In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install chromadb

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb


class EmbeddingFunction:
    def __init__(self):
        self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

    def __call__(self, input):
        return self.model.encode(input).tolist()


class DB:

    def __init__(self, distance_function, root_path):
        self.ef = EmbeddingFunction()
        self.client = chromadb.PersistentClient(path=root_path)
        self.distance_function = distance_function
        assert distance_function in ["l2", "ip", "cosine"], "Distance function should be 'l2' or 'ip' or 'cosine'"
        self.collection = self.client.get_or_create_collection("lab5_" + self.distance_function,
                                                               metadata={"hnsw:space": self.distance_function},
                                                               embedding_function=self.ef)

    def add(self, items):
        old_batch = 0
        new_batch = 1000
        while True:
            if new_batch > len(items["fragments"]):
                break
            self.collection.add(
                documents=items["fragments"][old_batch:new_batch],
                metadatas=items["metadata"][old_batch:new_batch],
                ids=items["ids"][old_batch:new_batch])
            old_batch = new_batch
            new_batch += 1000
        self.collection.add(
            documents=items["fragments"][old_batch:],
            metadatas=items["metadata"][old_batch:],
            ids=items["ids"][old_batch:])

    def query(self, query, n_results):
        return self.collection.query(query_embeddings=self.ef(query), n_results=n_results)

    def clear(self):
        self.client.delete_collection("lab5_" + self.distance_function)
        self.collection = self.client.get_or_create_collection("lab5_" + self.distance_function,
                                                               metadata={"hnsw:space": self.distance_function},
                                                               embedding_function=self.ef)

In [ ]:
database_l2 = DB("l2", "../lab_5/db")

In [ ]:
!pip install ctransformers
!pip install tiktoken
!pip install openai
!pip install cohere
!pip install kaleido
!pip install gradio==3.48.0

In [ ]:
from ctransformers import AutoModelForCausalLM
import gradio as gr

In [ ]:
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-OpenOrca-GGUF",
                                           model_file="mistral-7b-openorca.Q4_K_M.gguf",
                                           model_type="mistral")

In [ ]:
def process_request(message, history):
    context = "\n".join(database_l2.query(message, 5)["documents"][0])
    prompt = "Answer the question using provided context. Your answer should be in your own words and be no longer than 50 words.\n"
    prompt += "Context: " + context + "\n"
    # prompt += f"Previous chat history: {history}"
    prompt += "Question: " + message + "\n"
    prompt += "Answer: "
    answer = llm(prompt)
    return f"{prompt}\n\n{answer}"

Список вопросов:
1. Какова столица Франции?
2. Какие цвета присутствуют на флаге США?
3. Какой год начала Второй мировой войны?
4. Кто написал "Гарри Поттера"?
5. Как называется научная дисциплина, изучающая звезды и планеты?
6. Сколько дней в году?
7. Как называется самая высокая гора в мире?
8. Кто изображен на американской десятидолларовой купюре?
9. Как называется химический элемент с атомным номером 79?
10. Какой континент является самым крупным по площади?

In [ ]:
demo = gr.ChatInterface(fn=process_request, title="Chat bot")